Q31) 문자열, 형 변환

In [11]:
import pandas as pd
df = pd.read_csv("delivery_time.csv")

In [21]:
# 문자 슬라이싱
df['user_int'] = df['user'].str[5:]

In [23]:
# 자료형 변환
df['user_int'] = df['user_int'].astype(int)

In [25]:
# 합계
print(df['user_int'].sum())

261387


Q32) 합계(열 방향), 상위 값 선택

In [2]:
import pandas as pd
df = pd.read_csv("school_data.csv")

In [4]:
# 수학, 영어, 국어 점수 합계
df['total_score'] = df[['수학','영어','국어']].sum(axis=1)

In [8]:
# 합계 점수를 기준으로 상위 10명 선택
top10 = df.nlargest(10, 'total_score')

In [12]:
top10

,이름,수학,영어,국어,수학교사,영어교사,국어교사,total_score
6,원숭이,86,97,71,김선생,장선생,이선생,254
4,호랑이,83,92,75,박선생,장선생,이선생,250
8,코끼리,96,94,43,김선생,유선생,이선생,233
1,고양이,92,48,80,김선생,장선생,이선생,220
13,하이에나,96,40,84,김선생,장선생,최선생,220
18,독수리,96,34,81,박선생,장선생,이선생,211
28,돌고래,84,52,67,박선생,유선생,최선생,203
15,강치,83,69,49,박선생,유선생,이선생,201
24,참새,2,100,91,김선생,유선생,최선생,193
7,기린,97,85,1,김선생,유선생,최선생,183


In [10]:
# 선택된 10명의 수학 평균 점수 계산
result = top10['수학'].mean()
print(round(result))

82


Q33) 데이터프레임 재구조화

In [15]:
import pandas as pd
df = pd.read_csv("school_data.csv")

In [17]:
# 이름을 유지하면서 수학, 영어, 국어 형태 변경
melted_df = df.melt(id_vars=['이름'], value_vars=['수학', '영어', '국어'])

In [19]:
melted_df

,이름,variable,value
0,강아지,수학,66
1,고양이,수학,92
2,토끼,수학,98
3,사자,수학,17
4,호랑이,수학,83
...,...,...,...
85,타조,국어,49
86,개구리,국어,86
87,펠리칸,국어,3
88,돌고래,국어,67


In [21]:
# 가장 작은 값 25개의 합
result = melted_df['value'].nsmallest(25).sum()
print(result)

420


Q34) 데이터 합치기(concat)

In [24]:
import pandas as pd
df = pd.read_csv("school_data.csv")
df_science = pd.read_csv("school_data_science.csv")

In [26]:
# 두 데이터프레임 합치기(axis=1)
df = pd.concat([df, df_science], axis=1)

In [30]:
# 수평(열)으로 평균 계산
df['수영국과 평균'] = df[['수학','영어','국어','과학']].mean(axis=1)

In [32]:
# 수학, 영어, 국어, 과학 평균이 60점 이상
result = sum(df['수영국과 평균']>=60)
print(result)

9


Q35) 데이터 합치기(merge)

In [35]:
import pandas as pd
df = pd.read_csv("school_data.csv")
df_social = pd.read_csv("school_data_social.csv")

In [37]:
# '이름'을 기준으로 두 데이터프레임 합치기
merged_df = pd.merge(df, df_social, on='이름')

In [39]:
# 교사 필터링
cond1 = merged_df['영어교사']=='장선생'
cond2 = merged_df['사회교사']=='오선생'

In [41]:
# 필터링된 데이터에서 수학 점수 합
result = merged_df[cond1 & cond2]['수학'].sum()
print(result)

602


Q36) 조건별 변환(transform)

In [47]:
import pandas as pd
df=pd.read_csv("sales.csv")

In [49]:
# 지역별 평균 판매금액 계산 후, 결측값 대체
m=df.groupby('지역코드')['판매금액'].transform('mean')
df['판매금액']=df['판매금액'].fillna(m)

In [53]:
# 각 거래별 해당 지역의 평균 판매금액 구하기
df['지역평균'] = df.groupby('지역코드')['판매금액'].transform('mean')

In [55]:
# 판매금액과 지역평균의 차이(절대값) 계산하여 새로운 컬럼 생성
df['차이'] = abs(df['판매금액']-df['지역평균'])

In [57]:
# 지역별 차이 평균 계산 후, 최대값을 가지는 지역코드 출력
result = df.groupby('지역코드')['차이'].mean().idxmax()
result

103

Q37) 재구조화(unstack), 맵핑

In [65]:
import pandas as pd
df = pd.read_csv("store_sales.csv")

In [67]:
# 매출액 계산
df['매출액']=df['판매수량'] * df['단가']

In [71]:
day_mapping = {'월':'평일', '화':'평일', '수':'평일', '목':'평일', '금':'평일', '토':'주말', '일':'주말'}

In [73]:
# 매핑을 활용하여 새로운 열 추가
df['구분']=df['요일'].map(day_mapping)

In [75]:
# 매장코드 & 평일/주말별 매출액 합계
store_sales = df.groupby(['매장코드','구분'])['매출액'].sum().unstack()

In [77]:
# 평일과 주말 매출액 차이 계산
store_sales['차이'] = abs(store_sales['평일']-store_sales['주말'])

In [79]:
# 전체 매장 중 매출액 차이가 가장 큰 매장의 차이 값 찾기
store_sales['차이'].max()

241975

Q38) 피벗테이블

In [84]:
import pandas as pd
df = pd.read_csv("region_sales.csv")

In [86]:
# 피벗 테이블 생성 : Region, Channel별로 Product별 Sales 합계 계산
pivot_table = pd.pivot_table(df, index=['Region', 'Channel'], columns='Product', values='Sales', aggfunc='sum')

In [88]:
# A비율 계산: 제품 A의 판매액 / 총매출
pivot_table['총매출'] = pivot_table['A'] + pivot_table['B']
pivot_table['A비율'] = pivot_table['A'] / pivot_table['총매출']

In [90]:
# 최대 A비율 값 출력
result = pivot_table['A비율'].max()
round(result, 2)

0.51

Q39) 재구조화(melt), 그룹핑

In [93]:
import pandas as pd
df = pd.read_csv('monthly_sales.csv')

# melt로 Wide(가로) -> Long(세로) 변환
melted_df = pd.melt(
    df,
    id_vars='Region',
    value_vars=['Jan','Feb','Mar'],
    var_name='Month',
    value_name='Sales'
)

In [95]:
# Region, Month별 매출 합계 구하기
grouped_sales = melted_df.groupby(['Region', 'Month'])['Sales'].sum().reset_index()

In [97]:
# 매출 합계가 1400을 초과하는 건수
cond = grouped_sales['Sales']>1400
len(grouped_sales[cond])

6